In [1]:
#FIXME: REWRITE to the latest RAW corpuses format!

In [2]:
#!/usr/bin/env python
# coding: utf8
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from tqdm.auto import tqdm

In [3]:
CFG = {'device': 0, 'cpu_count': 4}
TESTS = False
spacy.require_gpu()

True

In [4]:
import gzip, json
def load_entries(fn): # '../data/datasets/nerus.jsonl.gz'
    entries = []
    with gzip.open(fn, 'r') as f:
        for line in tqdm(f):
            entry = json.loads(line)
            entries.append(entry)
    return entries
    #del entries

In [5]:
# from utils.corpus import Corpus

In [6]:
# NERUS_ds_test = load_entries('../../data/datasets/nerus_test.jsonl.gz')
# NERUS_ds_train = load_entries('../../data/datasets/nerus_train.jsonl.gz')
# NERUS = Corpus.from_raw('ru', NERUS_ds_train, NERUS_ds_test)

In [8]:
class Corpus:
    ents = {}
    ds_train = []
    ds_test = []

In [9]:
NERUS = Corpus()
NERUS.ents = {'ORG', 'PER', 'LOC'}
NERUS.ds_test = load_entries('../../data/datasets/nerus_test.jsonl.gz')
NERUS.ds_train = load_entries('../../data/datasets/nerus_train.jsonl.gz')

# BUGFIX for v0.5
# NERUS.ds_test = [{'raw': x['raw'], 'entities': x['entries']} for x in tqdm(NERUS.ds_test)]
# NERUS.ds_train = [{'raw': x['raw'], 'entities': x['entries']} for x in tqdm(NERUS.ds_train)]

# BUGFIX for v0.9
NERUS.ner = NERUS.ents

In [10]:
CORPORA = [NERUS]

In [11]:
for c in CORPORA:
    print(len(c.ner), len(c.ds_train), len(c.ds_test))

3 719295 20000


In [12]:
def get_other_pipes(nlp, x):
    return [pipe for pipe in nlp.pipe_names if pipe != "ner"]

In [13]:
def pluck(dict_list, attr):
    r = []
    for e in dict_list:
        r.append(e[attr])
    return r

def pluck_list(dict_list, *attrs):
    r = []
    for e in dict_list:
        r.append([e[a] for a in attrs])
    return r

def pluck_dict(dict_list, *attrs):
    r = []
    for e in dict_list:
        r.append({a: e[a] for a in attrs})
    return r

In [14]:
# def setup_model():
#     print("Creating new model...")
#     nlp = spacy.blank('ru')
#     return nlp
# nlp = setup_model()
# from tokenizer import set_stemming_tokenizer
# set_stemming_tokenizer(nlp)

In [15]:
USE_RU2 = False
if USE_RU2:
    import ru2e
    nlp = ru2e.load_ru2('../../ru2_pos_dep_stemming')
else:
    nlp = spacy.load('../../ru2')
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f2abb223550>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f2abb1bf5e8>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f2abb1bf648>)]

In [16]:
def add_ner(nlp, labels, rebuild=True):
    if 'ner' in nlp.pipe_names and rebuild:
        nlp.disable_pipes('ner')
    if 'ner' not in nlp.pipe_names:
        print("Creating new NER...")
        nlp_ner = nlp.create_pipe('ner')
        nlp.add_pipe(nlp_ner)
    else:
        print("Using existing NER...")
        nlp_ner = nlp.get_pipe('ner')
    print(nlp.pipeline)
    for l in labels:
        nlp_ner.add_label(l)
    assert set(labels) <= set(nlp_ner.labels)
    return nlp_ner
nlp_ner = add_ner(nlp, [l for c in CORPORA for l in c.ents], rebuild=True)
print(nlp_ner.labels)

assert nlp.get_pipe('ner') == nlp_ner

Creating new NER...
[('tagger', <spacy.pipeline.pipes.Tagger object at 0x7f2abb223550>), ('parser', <spacy.pipeline.pipes.DependencyParser object at 0x7f2abb1bf5e8>), ('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x7f2abb1bf648>)]
('PER', 'ORG', 'LOC')


In [17]:
from utils.tqdm import tqdm_batches

In [18]:
def unseen_names(nlp, unseen):
    ner = nlp.get_pipe('ner')
    return [mt for i, mt in enumerate(ner.move_names) if i in unseen]
    
def enable_all_entities(nlp):
    ner = nlp.get_pipe('ner')
    ner.model.unseen_classes = set()

def enable_entities(nlp, labels):
    ner = nlp.get_pipe('ner')
    # print("Unseen classes were:", unseen_names(nlp, ner.model.unseen_classes))
    unseen = set()
    for i, mt in enumerate(ner.move_names):
        if '-' in mt:
            l = mt.split('-', 1)[1]
            # print(mt, l)
            if l not in labels:
                unseen.add(i)
    # print("Set unseen classes to:", unseen_names(nlp, unseen))
    ner.model.unseen_classes = unseen

if TESTS:
    print('Training', KR.ents)
    enable_entities(nlp, KR.ents)

In [19]:
import spacy
import random
from spacy.gold import GoldParse
from spacy.scorer import Scorer
import pandas
pandas.set_option('display.precision', 3) 

def _evaluate(model, batches):
    if model.get_pipe('ner').model is True:
        print("Initializing model!")
        model.begin_training(**CFG)
    scorer = Scorer()
    for batch in batches:
        docs = pluck(batch, 'raw')
        docs = model.pipe(docs)
        for doc, parse in zip(docs, batch):
            scorer.score(doc, GoldParse(doc, entities=parse['entities']))
    return scorer.scores

def evaluate(model, dataset, batch_size=32):
    batches = tqdm_batches(minibatch(dataset, batch_size), total=len(dataset), leave=False)
    return _evaluate(model, batches)

def evaluate_data_source(model, ds, count=None, batch_size=32):
    enable_entities(model, ds.ents)
    if count:
        dataset = random.sample(ds.ds_test, count)
    else:
        dataset = ds.ds_test
    res = evaluate(model, dataset)
    return {k:v for k,v in res['ents_per_type'].items() if k in ds.ents}

def display_ents(list_of_scores):
    display(pandas.DataFrame.from_records(list_of_scores).T)

if TESTS or 0:
    res = evaluate(nlp, NERUS.ds_test, 1000)
    print(pluck_dict([res], 'token_acc', 'tags_acc', 'uas', 'las'))
    display_ents(res['ents_per_type'])

if TESTS or 0:
    scores = evaluate_data_source(nlp, NERUS, count=1000, batch_size=32)
    display_ents(scores)

In [20]:
import numpy

def _train_epoch(model, labels, batches):
    with model.disable_pipes(*get_other_pipes(nlp, 'ner')):
        if model.get_pipe('ner').model is True:
            print("Initializing model!")
            model.begin_training(**CFG)
        optimizer = model.resume_training(**CFG)
        losses = {}
        n_docs = 0
        for batch in batches:
            texts = pluck(batch, 'raw')
            anns = pluck_dict(batch, 'entities')
            enable_entities(model, labels)
            model.update(texts, anns, drop=0.2, losses=losses, sgd=optimizer)
            n_docs += len(batch)
        meta = {
            'docs': n_docs,
            'loss': {k: numpy.log(1e-10 + (v / n_docs)) for k,v in losses.items()},
        }
    enable_all_entities(model)
    return meta

def train_epoch(model, ds, batch_size, count=None):
    if count is None:
        dataset = ds.ds_train.copy()
        random.shuffle(dataset)
    else:
        dataset = random.sample(ds.ds_train, count)
    batches = minibatch(dataset, size=size_)
    _train_epoch(nlp, ds.ents, tqdm_batches(batches, total=len(dataset)))

if TESTS or 0:
    size_ = compounding(1., 32., 1.001)
    train_epoch(nlp, NERUS, batch_size=size_, count=1000)

if TESTS or 0:
    size_ = compounding(1., 32., 1.001)
    train_epoch(nlp, KR, batch_size=size_, count=1000)

if TESTS or 0:
    res = {}
    for c in CORPORA:
        res.update(evaluate_data_source(nlp, c, 1000))
    display_ents(res)

In [21]:
#     res = {}
#     for c in CORPORA:
#         res.update(evaluate_data_source(nlp, c, 1000))
#     display_ents(res)

In [22]:
size_ = compounding(1., 32., 1.0005)
for e in tqdm(range(60)):
    for ds_train in CORPORA:
        train_epoch(nlp, ds_train, batch_size=size_, count=1000) # 100000
    res = {}
    for c in CORPORA:
        res.update(evaluate_data_source(nlp, c, 500)) # 5000
    display_ents(res)

Initializing model!


KeyboardInterrupt: 

In [ ]:
optimizer = nlp.resume_training(**CFG)
with nlp.use_params(optimizer.averages):
    res = evaluate_data_source(nlp, c, count=None)
    display_ents(res)

In [ ]:
optimizer.averages

In [ ]:
import spacy.displacy
def view_example(nlp, s):
    print('Text:', s['raw'])
    doc = nlp(s['raw'])
#     print("Actual:", [(e, e.label_) for e in doc.ents])
    print("Expected:", [(s['raw'][a:b],c,a,b) for a,b,c in s['entities']])
    spacy.displacy.render(doc, style='ent')

enable_all_entities(nlp)

for s in NERUS.ds_test[:2]:
    view_example(nlp, s)
# for s in KR.ds_test[:3]:
#     view_example(nlp, s)

In [ ]:
from pathlib import Path
# save model to output directory
def save_model(nlp, output_dir):
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
save_model(nlp, '../../ru2_ner')

In [ ]:
nlp.tokenizer('приветы всем'.split())

In [ ]:
spacy.displacy.render(nlp('Вчера мы с Васей пошли гулять по СПб, Гаванская ул. 18'), 
                      style='ent')